* References 
1. [Reading and Writing the Apache Parquet Format(Aparch Arrow Documentation|)](https://arrow.apache.org/docs/python/parquet.html)

In [1]:
# requisite
!pip install pyarrow

You should consider upgrading via the 'C:\Users\seoun\anaconda3\python.exe -m pip install --upgrade pip' command.


# parguet, avro, csv comparative test 

In [2]:
# make dataset for comparison
from sklearn.datasets import make_classification
import pandas as pd 

n_samples = int(1e+6)
num_X = 29

X, y = make_classification(
    n_samples=n_samples, 
    n_features=num_X    
)

df = pd.DataFrame(X, columns=[f"col_{int(i+1)}" for i in range(num_X)])
df['y'] = y

In [3]:
df

,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,...,col_21,col_22,col_23,col_24,col_25,col_26,col_27,col_28,col_29,y
0,-2.269917,-0.548390,-1.304414,-0.803364,0.389434,0.061894,0.371249,0.780031,1.505127,-0.164102,...,-0.788797,1.158041,1.879119,0.555265,1.223207,-0.997294,-0.161569,0.998925,-1.299078,1
1,-1.277575,-1.199441,1.101118,1.698404,-0.385637,0.546726,-0.178638,-0.054332,0.878290,3.624857,...,0.679446,-0.916311,1.883561,-0.568607,-0.004924,-0.455516,-0.697243,-0.224506,-0.419221,1
2,-0.197409,0.905835,-0.118557,-0.018829,-1.294973,0.015264,0.910070,-0.923833,1.318269,0.910617,...,0.340163,0.499457,-0.868820,1.445217,-0.573818,-0.384927,1.235920,-1.161241,0.114517,0
3,-0.438786,0.895868,1.703515,-1.187641,0.352891,0.911864,-0.329896,0.828690,-0.121513,1.824285,...,-0.560867,0.211777,-1.051707,-0.006396,0.681823,0.125102,1.079078,-0.921679,-1.625561,0
4,-0.096392,-1.178414,-1.564027,-0.036047,-1.258260,0.290279,0.069543,0.985999,0.741983,0.759746,...,-0.944363,1.120809,-1.273784,-0.321723,0.783448,-0.533004,-0.366465,0.277151,1.053440,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,0.501364,0.466687,0.978387,0.290811,0.408511,0.831120,0.195551,-0.587097,1.176899,1.162911,...,1.741641,0.984381,0.507181,0.432752,1.984266,-0.467804,-0.108557,0.133221,0.495939,1
999996,0.773677,0.485538,-1.742684,-0.113992,-0.191080,0.247876,-0.199901,0.801454,-1.198073,0.451386,...,-1.015265,1.995838,-0.102721,1.227792,0.262236,-0.347495,-0.619685,-0.985195,-0.347435,1
999997,-0.360279,1.766277,-1.091394,0.752158,0.745229,-0.968460,-0.240746,-0.966334,0.615075,0.241806,...,0.124188,-0.878545,-0.875173,0.280072,0.116220,-0.148509,0.706903,0.997562,-1.407469,1
999998,-0.173365,-0.203906,-1.157908,0.702131,-0.899072,-0.772750,-0.499171,1.548589,-0.189970,0.999821,...,0.534505,1.107877,0.263203,0.446082,-0.141998,0.085071,-1.017549,-1.655263,0.152282,0


In [4]:
# Write pd.DataFrame to Parquet, csv format 
import pyarrow as pa
import pyarrow.parquet as pq

# to csv
df.to_csv('example.csv')

# to parquet
table = pa.Table.from_pandas(df)    # Convert pd.DataFrame to parquet format
pa.parquet.write_table(table, 'example.parquet')    # Write pd.DataFrame to Parquet format 

## 01. Read test

* Whole data

In [5]:
%%timeit 
pd.read_csv('example.csv', index_col=0)

11.7 s ± 150 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%%timeit 
pq.read_table('example.parquet').to_pandas()

498 ms ± 53.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
pd.read_parquet('example.parquet')

489 ms ± 16.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


* With selected columns 

In [8]:
%%timeit 
pd.read_csv('example.csv', usecols=['col_1', 'col_5', 'col_10', 'y'])

5.23 s ± 53.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%%timeit 
pq.read_table('example.parquet', columns=['col_1', 'col_5', 'col_10', 'y']).to_pandas()

57.9 ms ± 1.74 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
%%timeit
pd.read_parquet('example.parquet', columns=['col_1', 'col_5', 'col_10', 'y'])

61.6 ms ± 1.33 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
